In [232]:
# import libraries
import numpy as np
import pandas as pd

In [233]:
# Creates file name/path
file_name = "Calculating Median of Cash Withdrawal with Excel.xlsx"

In [234]:
# Create pandas dataframe
df = pd.read_excel(file_name, sheet_name="Sheet1", skiprows=1, header=0, index_col=None, usecols="A:C")
df

,Customer Id,Transaction Amount,Transaction Date
0,100,19,2023-08-25
1,100,8,2023-08-27
2,100,7,2023-08-29
3,100,16,2023-08-31
4,100,3,2023-09-03
5,100,5,2023-09-05
6,100,2,2023-09-06
7,101,13,2023-08-25
8,101,22,2023-08-27
9,101,5,2023-08-29


In [235]:
# Creating Transaction Count for every customer. We wil use this skip first 2 calculations
df["Transaction Count"] = df.groupby('Customer Id').cumcount() + 1
df

,Customer Id,Transaction Amount,Transaction Date,Transaction Count
0,100,19,2023-08-25,1
1,100,8,2023-08-27,2
2,100,7,2023-08-29,3
3,100,16,2023-08-31,4
4,100,3,2023-09-03,5
5,100,5,2023-09-05,6
6,100,2,2023-09-06,7
7,101,13,2023-08-25,1
8,101,22,2023-08-27,2
9,101,5,2023-08-29,3


In [236]:
# Calculationg median of 3 transactions.

# Sort the DataFrame
df = df.sort_values(by=['Customer Id', 'Transaction Date'])

# Reset the index to ensure proper alignment
df.reset_index(drop=True, inplace=True)

# Calculate the rolling median for each customer using groupby and rolling
window_size = 3  # Window size of 3 for current row, previous row, and two rows above (last 3 transactions)

# Calculating median of last 3 transaction of each customer regarding. window_size determines the number of transactions to include.
rolling_medians = df.groupby('Customer Id')['Transaction Amount'].rolling(window=window_size, min_periods=3).median()
# min_periods = Minimum number of observations in window required to have a value; otherwise, result is np.nan. If it is different than window_size (for this use case) it still can calculate even number of transactions less than 3.

# Reset the index of the rolling_medians DataFrame to align with df
rolling_medians = rolling_medians.reset_index(level=0, drop=True)

# Assign the calculated rolling medians to the DataFrame
df['Median of N Values'] = rolling_medians

# Replace NaN values in "Median of N Values" with 0 because we will calculate "Double of Transaction" column based on this column later. NaN values cause a (minor) problem.
df['Median of N Values'].fillna(0, inplace=True)

df

,Customer Id,Transaction Amount,Transaction Date,Transaction Count,Median of N Values
0,100,19,2023-08-25,1,0.0
1,100,8,2023-08-27,2,0.0
2,100,7,2023-08-29,3,8.0
3,100,16,2023-08-31,4,8.0
4,100,3,2023-09-03,5,7.0
5,100,5,2023-09-05,6,5.0
6,100,2,2023-09-06,7,3.0
7,101,13,2023-08-25,1,0.0
8,101,22,2023-08-27,2,0.0
9,101,5,2023-08-29,3,13.0


In [237]:
# Create a new column "Double of Transaction" based on the conditions (If median = 0 then it is 0, otherwise it is multiplication of "Transaction Amount" by 2.
df['Double of Transaction'] = df['Transaction Amount'] * 2
df.loc[df['Median of N Values'] == 0, 'Double of Transaction'] = 0

df

,Customer Id,Transaction Amount,Transaction Date,Transaction Count,Median of N Values,Double of Transaction
0,100,19,2023-08-25,1,0.0,0
1,100,8,2023-08-27,2,0.0,0
2,100,7,2023-08-29,3,8.0,14
3,100,16,2023-08-31,4,8.0,32
4,100,3,2023-09-03,5,7.0,6
5,100,5,2023-09-05,6,5.0,10
6,100,2,2023-09-06,7,3.0,4
7,101,13,2023-08-25,1,0.0,0
8,101,22,2023-08-27,2,0.0,0
9,101,5,2023-08-29,3,13.0,10


In [238]:
# Create a new column "Alert Status" based on the condition (result column)
df['Alert Status'] = np.where(
    (df['Median of N Values'] > 0) & 
    (df['Double of Transaction'] > 0) & 
    (df['Median of N Values'] >= df['Double of Transaction']), 
    'SEND ALERT', 
    'DO NOT SEND ALERT'
)

df

,Customer Id,Transaction Amount,Transaction Date,Transaction Count,Median of N Values,Double of Transaction,Alert Status
0,100,19,2023-08-25,1,0.0,0,DO NOT SEND ALERT
1,100,8,2023-08-27,2,0.0,0,DO NOT SEND ALERT
2,100,7,2023-08-29,3,8.0,14,DO NOT SEND ALERT
3,100,16,2023-08-31,4,8.0,32,DO NOT SEND ALERT
4,100,3,2023-09-03,5,7.0,6,SEND ALERT
5,100,5,2023-09-05,6,5.0,10,DO NOT SEND ALERT
6,100,2,2023-09-06,7,3.0,4,DO NOT SEND ALERT
7,101,13,2023-08-25,1,0.0,0,DO NOT SEND ALERT
8,101,22,2023-08-27,2,0.0,0,DO NOT SEND ALERT
9,101,5,2023-08-29,3,13.0,10,SEND ALERT
